In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fin-tweets-training/fin_tweets_dataset (1).csv
/kaggle/input/fin-tweets-validation/sent_valid.csv


In [4]:
from transformers import AutoTokenizer
from transformers import AutoTokenizer
from datasets import Dataset
import pandas as pd
df_train = pd.read_csv('/kaggle/input/fin-tweets-training/fin_tweets_dataset (1).csv')
df_valid = pd.read_csv('/kaggle/input/fin-tweets-validation/sent_valid.csv')
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

train_dataset = Dataset.from_pandas(df_train)
valid_dataset = Dataset.from_pandas(df_valid)

# Initialize the tokenizer
 
# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

# Apply tokenization
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_valid_dataset = valid_dataset.map(tokenize_function, batched=True)

# R

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [6]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 3  # Number of target labels
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=8,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_valid_dataset,
)

trainer.train()


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,No log,0.810858
2,No log,0.606032
3,No log,0.608758
4,No log,0.607193
5,No log,0.667129
6,No log,0.709621
7,No log,0.676072
8,No log,0.696993


TrainOutput(global_step=376, training_loss=0.39285655731850483, metrics={'train_runtime': 1065.5776, 'train_samples_per_second': 11.261, 'train_steps_per_second': 0.353, 'total_flos': 3157361012736000.0, 'train_loss': 0.39285655731850483, 'epoch': 8.0})

In [7]:
from huggingface_hub import notebook_login

notebook_login()

In [8]:
model.save_pretrained('./FINBERX')
tokenizer.save_pretrained('./FINBERX')


('./FINBERX/tokenizer_config.json',
 './FINBERX/special_tokens_map.json',
 './FINBERX/vocab.txt',
 './FINBERX/added_tokens.json',
 './FINBERX/tokenizer.json')

In [9]:
from huggingface_hub import HfApi, HfFolder, Repository
import os
import shutil

# Initialize HfApi instance
api = HfApi()

# Define your Hugging Face username and the name of your repository
hf_username = "zcharaf"  # Replace with your Hugging Face username
repo_name = "FINBERX"   # Replace with your desired repository name

# Create a new repository on the Hugging Face Hub
repo_url = api.create_repo(repo_name, token=HfFolder.get_token(), exist_ok=True)

# Local directories
source_dir = './FINBERX'         # The directory where your model files are saved
target_dir = './FINBERX_clone'   # The directory where you want to clone your repository

# Check if target directory exists and if it contains a .git directory
if os.path.exists(target_dir):
    if os.path.isdir(os.path.join(target_dir, '.git')):
        # If it's already a git repo, delete it
        shutil.rmtree(target_dir)

# Clone the repository
repo = Repository(local_dir=target_dir, clone_from=repo_url)
repo.git_pull()

# Copy files from the saved model to the cloned repository
if os.path.exists(source_dir) and os.listdir(source_dir):
    os.system(f'cp -r {source_dir}/* {target_dir}/')
    
    # Commit and push to the repository
    repo.git_add(auto_lfs_track=True)
    repo.git_commit("First commit with my fine-tuned model")
    repo.git_push()
else:
    print(f"Error: Source directory '{source_dir}' does not exist or is empty.")


/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/zcharaf/FINBERX into local empty directory.


Upload file model.safetensors:   0%|          | 1.00/418M [00:00<?, ?B/s]

To https://huggingface.co/zcharaf/FINBERX
   47678f1..6507b7a  main -> main

